In [5]:
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
import os

def create_rectangular_maze(width, height):
    # Create a grid filled with walls
    maze = np.ones((height*2+1, width*2+1))

    # Define the starting point
    x, y = (0, 0)
    maze[2*x+1, 2*y+1] = 0

    # Initialize the stack with the starting point
    stack = [(x, y)]
    while len(stack) > 0:
        x, y = stack[-1]

        # Define possible directions
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        random.shuffle(directions)

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if nx >= 0 and ny >= 0 and nx < height and ny < width and maze[2*nx+1, 2*ny+1] == 1:
                maze[2*nx+1, 2*ny+1] = 0
                maze[2*x+1+dx, 2*y+1+dy] = 0
                stack.append((nx, ny))
                break
        else:
            stack.pop()

    # Create an entrance and an exit
    maze[1, 0] = 0  # Entrance
    maze[-2, -1] = 0  # Exit

    # Define start and end points based on dimensions
    start_point = (1, 0)
    end_point = (height*2-1, width*2)

    return maze, start_point, end_point

def export_maze_to_excel(maze, filename):
    df = pd.DataFrame(maze)
    df.to_excel(filename, index=False, header=False)

# Create the directory if it doesn't exist
os.makedirs('maze', exist_ok=True)

# Generate 10 mazes with increasing dimensions
initial_width = 20
initial_height = 20

for i in range(10):
    width = initial_width + i
    height = initial_height + i
    maze, start_point, end_point = create_rectangular_maze(width, height)
    filename = f'{i}.xlsx'
    export_maze_to_excel(maze, filename)
    print(f'Maze {width}x{height} saved as {filename}')
    


Maze 20x20 saved as 0.xlsx
Maze 21x21 saved as 1.xlsx
Maze 22x22 saved as 2.xlsx
Maze 23x23 saved as 3.xlsx
Maze 24x24 saved as 4.xlsx
Maze 25x25 saved as 5.xlsx
Maze 26x26 saved as 6.xlsx
Maze 27x27 saved as 7.xlsx
Maze 28x28 saved as 8.xlsx
Maze 29x29 saved as 9.xlsx


In [ ]:
import pygame
import cv2
import sys
import os

# Initialize pygame
def initialize_pygame():
    pygame.init()

# Constants
VIDEO_DIR = "Video_Animations"
IMAGE_DIR = "Images"

INTRO_VIDEO = os.path.join(VIDEO_DIR, "Intro_Page.mp4")
MAIN_VIDEO = os.path.join(VIDEO_DIR, "Main_Page.mp4")
SINGLEPLAYER_VIDEO = os.path.join(VIDEO_DIR, "SinglePlayer_Page.mp4")
MULTIPLAYER_VIDEO = os.path.join(VIDEO_DIR, "Multiplayer_Page.mp4")

BUTTON_IMAGES = {
    "Start": os.path.join(IMAGE_DIR, "Start.png"),
    "Singleplayer": os.path.join(IMAGE_DIR, "SinglePlayer.png"),
    "Multiplayer": os.path.join(IMAGE_DIR, "MultiPlayer.png"),
    "Settings": os.path.join(IMAGE_DIR, "Setting.png"),
    "Quit Game": os.path.join(IMAGE_DIR, "Quit_Game.png"),
    "Return": os.path.join(IMAGE_DIR, "Back.png"),  # Renamed key but keeps the same image
    "Levels": os.path.join(IMAGE_DIR, "Levels.png"),
    "Free to play": os.path.join(IMAGE_DIR, "Free_to_Play.png")
}

BUTTON_HEIGHT = 100  # Standardized button height

def get_screen_dimensions():
    screen_info = pygame.display.Info()
    return screen_info.current_w, screen_info.current_h

def set_caption():
    pygame.display.set_caption("Game with Persistent Buttons")

def load_and_scale_button_images():
    button_images = {}
    for key, path in BUTTON_IMAGES.items():
        if os.path.exists(path):  # Ensure image exists before loading
            button_images[key] = scale_button_image(path)
        else:
            print(f"Warning: Missing image {path}")
    return button_images

def scale_button_image(image_path):
    image = pygame.image.load(image_path)
    aspect_ratio = image.get_width() / image.get_height()
    new_width = int(BUTTON_HEIGHT * aspect_ratio)
    return pygame.transform.scale(image, (new_width, BUTTON_HEIGHT))

def set_up_screen(width, height):
    return pygame.display.set_mode((width, height), pygame.FULLSCREEN)

def play_video_with_buttons(video_path, buttons_config, screen, screen_width, screen_height, button_images):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    clock = pygame.time.Clock()
    last_frame = None  # Store last valid frame

    while True:
        ret, frame = cap.read()
        if ret:
            last_frame = frame  # Update last valid frame
        elif last_frame is not None:
            frame = last_frame  # Keep displaying the last frame
        else:
            break  # If there's no valid frame, exit

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (screen_width, screen_height))
        frame_surface = pygame.surfarray.make_surface(frame.swapaxes(0, 1))
        screen.blit(frame_surface, (0, 0))

        # Button positions
        button_positions = {
            "Start": (530, 470),
            "Singleplayer": (50, 200),
            "Multiplayer": (50, 320),
            "Settings": (50, 440),
            "Quit Game": (50, 560),
            "Levels": (580, 200),
            "Return": (580, 320),
            "Free to play": (580, 440)
        }

        # Draw buttons
        buttons = []
        for label, image in buttons_config.items():
            if label in button_positions:
                rect = image.get_rect(topleft=button_positions[label])
                screen.blit(image, rect.topleft)
                buttons.append((label, rect))

        pygame.display.update()

        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                cap.release()
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                for label, rect in buttons:
                    if rect.collidepoint(event.pos):
                        if label == "Start":
                            cap.release()
                            play_video_with_buttons(MAIN_VIDEO, {  # FIX: Now starts main menu!
                                "Singleplayer": button_images["Singleplayer"],
                                "Multiplayer": button_images["Multiplayer"],
                                "Settings": button_images["Settings"],
                                "Quit Game": button_images["Quit Game"]
                            }, screen, screen_width, screen_height, button_images)
                        elif label == "Return":
                            cap.release()
                            return
                        elif label == "Singleplayer":
                            play_video_with_buttons(SINGLEPLAYER_VIDEO, {
                                "Return": button_images["Return"],  
                                "Levels": button_images["Levels"],
                                "Free to play": button_images["Free to play"]
                            }, screen, screen_width, screen_height, button_images)
                        elif label == "Multiplayer":
                            play_video_with_buttons(MULTIPLAYER_VIDEO, {
                                "Return": button_images["Return"]
                            }, screen, screen_width, screen_height, button_images)
                        elif label == "Settings":
                            print("Settings button clicked!")
                        elif label == "Quit Game":
                            if show_quit_popup(screen, screen_width, screen_height):
                                pygame.quit()
                                sys.exit()

        clock.tick(fps)  # Smooth frame rate

    cap.release()

def show_quit_popup(screen, screen_width, screen_height):
    """ Show a quit confirmation popup with Yes/No buttons. Returns True if Yes is clicked, False otherwise. """
    popup_width, popup_height = 400, 200
    popup_x = (screen_width - popup_width) // 2
    popup_y = (screen_height - popup_height) // 2

    font = pygame.font.Font(None, 40)
    yes_button = pygame.Rect(popup_x + 50, popup_y + 120, 120, 50)
    no_button = pygame.Rect(popup_x + 230, popup_y + 120, 120, 50)

    running = True
    while running:
        pygame.draw.rect(screen, (50, 50, 50), (popup_x, popup_y, popup_width, popup_height))  # Popup background
        pygame.draw.rect(screen, (255, 255, 255), yes_button)  # Yes button
        pygame.draw.rect(screen, (255, 255, 255), no_button)  # No button

        text = font.render("Quit Game?", True, (255, 255, 255))
        yes_text = font.render("Yes", True, (0, 0, 0))
        no_text = font.render("No", True, (0, 0, 0))

        screen.blit(text, (popup_x + 120, popup_y + 40))
        screen.blit(yes_text, (popup_x + 90, popup_y + 130))
        screen.blit(no_text, (popup_x + 270, popup_y + 130))

        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return True  # If window is closed, treat it as quitting
            if event.type == pygame.MOUSEBUTTONDOWN:
                if yes_button.collidepoint(event.pos):
                    return True  # Quit the game
                if no_button.collidepoint(event.pos):
                    return False  # Close popup

    return False  # Default to not quitting

def main():
    if not os.path.exists(INTRO_VIDEO) or not os.path.exists(MAIN_VIDEO):
        print(f"Error: Ensure {INTRO_VIDEO} and {MAIN_VIDEO} are in the same directory as the script.")
        sys.exit()

    initialize_pygame()
    screen_width, screen_height = get_screen_dimensions()
    set_caption()
    button_images = load_and_scale_button_images()
    screen = set_up_screen(screen_width, screen_height)

    # Play intro video with Start button
    play_video_with_buttons(INTRO_VIDEO, {"Start": button_images["Start"]}, screen, screen_width, screen_height, button_images)

if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.13.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\pkans\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


SystemExit: 

C:\Users\pkans\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import numpy as np
import pygame
import sys
import time as Clock
import random
import pandas as pd

# Load the maze from an Excel file
def load_maze_from_excel(filename):
    df = pd.read_excel(filename, header=None)
    return df.values

# Load the maze
filename = 'maze/9.xlsx'
maze = load_maze_from_excel(filename)
dim = (maze.shape[0] - 1) // 2

# Set the start and end points
start_point = (1, 0)
end_point = (maze.shape[0] - 2, maze.shape[1] - 1)

# Function to place items randomly in the maze
def place_items(num_items, maze, items_list):
    for _ in range(num_items):
        while True:
            x, y = random.randint(1, dim-1), random.randint(1, dim-1)
            pos = (2 * x + 1, 2 * y + 1)
            if maze[pos] == 0 and pos not in items_list and pos not in [start_point, end_point]:
                items_list.append(pos)
                break

# Place items in the maze
gold_coins, diamonds, bombs = [], [], []
place_items(random.choice([1, 2]), maze, gold_coins)
place_items(random.choice([1, 2]), maze, diamonds)
place_items(random.choice([1, 2]), maze, bombs)

# Initialize Pygame
pygame.init()
block_size = 20
VIEWPORT_WIDTH = 800
VIEWPORT_HEIGHT = 400
VIEWPORT_COLS = VIEWPORT_WIDTH // block_size
VIEWPORT_ROWS = VIEWPORT_HEIGHT // block_size

screen = pygame.display.set_mode((VIEWPORT_WIDTH, VIEWPORT_HEIGHT))
pygame.display.set_caption("Maze Game")

gold_coin_img = pygame.image.load('Images/coin.png').convert_alpha()
diamond_img = pygame.image.load('Images/diamond.png').convert_alpha()
gold_coin_img = pygame.transform.scale(gold_coin_img, (block_size, block_size))
diamond_img = pygame.transform.scale(diamond_img, (block_size, block_size))

# Game loop variables
player_pos = list(start_point)
running = True
clock = pygame.time.Clock()
viewport_x, viewport_y = 0, 0
start_time = Clock.time()
score = 100000

while running:
    elapsed_time = Clock.time() - start_time
    score = 100000 - int(elapsed_time) * 1000 
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            new_pos = list(player_pos)
            if event.key in (pygame.K_w, pygame.K_UP) and maze[player_pos[0] - 1, player_pos[1]] == 0:
                new_pos[0] -= 1
            elif event.key in (pygame.K_s, pygame.K_DOWN) and maze[player_pos[0] + 1, player_pos[1]] == 0:
                new_pos[0] += 1
            elif event.key in (pygame.K_a, pygame.K_LEFT) and maze[player_pos[0], player_pos[1] - 1] == 0:
                new_pos[1] -= 1
            elif event.key in (pygame.K_d, pygame.K_RIGHT) and maze[player_pos[0], player_pos[1] + 1] == 0:
                new_pos[1] += 1
            player_pos = new_pos

    # Check for item collection
    if tuple(player_pos) in gold_coins:
        gold_coins.remove(tuple(player_pos))
        score *= 2
    if tuple(player_pos) in diamonds:
        diamonds.remove(tuple(player_pos))
        score *= 3
    if tuple(player_pos) in bombs:
        bombs.remove(tuple(player_pos))
        score -= 100000
    
    viewport_x = max(0, min(player_pos[1] - VIEWPORT_COLS // 2, maze.shape[1] - VIEWPORT_COLS))
    viewport_y = max(0, min(player_pos[0] - VIEWPORT_ROWS // 2, maze.shape[0] - VIEWPORT_ROWS))

    screen.fill((255, 255, 255))
    
    for i in range(viewport_y, min(viewport_y + VIEWPORT_ROWS, maze.shape[0])):
        for j in range(viewport_x, min(viewport_x + VIEWPORT_COLS, maze.shape[1])):
            color = (0, 0, 0) if maze[i, j] == 1 else (255, 255, 255)
            pygame.draw.rect(screen, color, pygame.Rect((j - viewport_x) * block_size, (i - viewport_y) * block_size, block_size, block_size))

    for gold_coin in gold_coins:
        if viewport_y <= gold_coin[0] < viewport_y + VIEWPORT_ROWS and viewport_x <= gold_coin[1] < viewport_x + VIEWPORT_COLS:
            screen.blit(gold_coin_img, ((gold_coin[1] - viewport_x) * block_size, (gold_coin[0] - viewport_y) * block_size))

    for diamond in diamonds:
        if viewport_y <= diamond[0] < viewport_y + VIEWPORT_ROWS and viewport_x <= diamond[1] < viewport_x + VIEWPORT_COLS:
            screen.blit(diamond_img, ((diamond[1] - viewport_x) * block_size, (diamond[0] - viewport_y) * block_size))

    pygame.draw.rect(screen, (0, 0, 255), pygame.Rect((player_pos[1] - viewport_x) * block_size, (player_pos[0] - viewport_y) * block_size, block_size, block_size))

    # Display score in the center of the screen
    font = pygame.font.Font(None, 36)
    score_text = font.render(f"Score: {score}", True, (0, 0, 0))
    text_rect = score_text.get_rect(center=(VIEWPORT_WIDTH // 2, 20))
    screen.blit(score_text, text_rect)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()
sys.exit()


SystemExit: 

C:\Users\pkans\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import socket
import threading
import json
import random
import numpy as np

# Multiplayer game state
rooms = {}

# Function to create a random maze
def create_rectangular_maze(width, height):
    maze = np.ones((height * 2 + 1, width * 2 + 1))
    x, y = (0, 0)
    maze[2 * x + 1, 2 * y + 1] = 0
    stack = [(x, y)]
    
    while stack:
        x, y = stack[-1]
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        random.shuffle(directions)
        
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < height and 0 <= ny < width and maze[2 * nx + 1, 2 * ny + 1] == 1:
                maze[2 * nx + 1, 2 * ny + 1] = 0
                maze[2 * x + 1 + dx, 2 * y + 1 + dy] = 0
                stack.append((nx, ny))
                break
        else:
            stack.pop()

    # Define start and end points
    start_point = (1, 0)
    end_point = (height * 2 - 1, width * 2)

    return maze.tolist(), start_point, end_point

# Handle each client
def handle_client(conn, addr, room_code, player_id):
    global rooms
    conn.sendall(json.dumps(rooms[room_code]["game_state"]).encode())

    while True:
        try:
            data = conn.recv(1024).decode()
            if not data:
                break
            move_data = json.loads(data)

            if "move" in move_data:
                move = move_data["move"]
                px, py = rooms[room_code]["game_state"]["players"][player_id]["pos"]

                # Movement logic
                if move == "UP" and rooms[room_code]["maze"][px - 1][py] == 0:
                    px -= 1
                elif move == "DOWN" and rooms[room_code]["maze"][px + 1][py] == 0:
                    px += 1
                elif move == "LEFT" and rooms[room_code]["maze"][px][py - 1] == 0:
                    py -= 1
                elif move == "RIGHT" and rooms[room_code]["maze"][px][py + 1] == 0:
                    py += 1

                rooms[room_code]["game_state"]["players"][player_id]["pos"] = (px, py)

                # Check if player reached the end
                if (px, py) == rooms[room_code]["game_state"]["end_point"]:
                    rooms[room_code]["game_state"]["message"] = "You lost!" if player_id == "player1" else "You won!"
                    break

                # Send updated game state to both players
                for player in rooms[room_code]["connections"]:
                    rooms[room_code]["connections"][player].sendall(json.dumps(rooms[room_code]["game_state"]).encode())

        except Exception as e:
            print(f"Error with {addr}: {e}")
            break

    conn.close()

# Start the server
def start_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(("0.0.0.0", 5555))
    server.listen()

    print("Server started on port 5555")

    while True:
        conn, addr = server.accept()
        data = json.loads(conn.recv(1024).decode())

        if data["action"] == "create":
            room_code = str(random.randint(1000, 9999))
            width, height = random.randint(10, 15), random.randint(10, 15)
            maze, start_point, end_point = create_rectangular_maze(width, height)

            rooms[room_code] = {
                "game_state": {
                    "maze": maze,
                    "players": {
                        "player1": {"pos": start_point},
                    },
                    "gold_coins": [(random.randint(1, width * 2 - 1), random.randint(1, height * 2 - 1)) for _ in range(5)],
                    "diamonds": [(random.randint(1, width * 2 - 1), random.randint(1, height * 2 - 1)) for _ in range(3)],
                    "end_point": end_point
                },
                "connections": {"player1": conn}
            }

            conn.sendall(json.dumps({"room_code": room_code, "game_state": rooms[room_code]["game_state"]}).encode())

        elif data["action"] == "join":
            room_code = data["room_code"]
            if room_code in rooms and "player2" not in rooms[room_code]["game_state"]["players"]:
                rooms[room_code]["game_state"]["players"]["player2"] = {"pos": rooms[room_code]["game_state"]["players"]["player1"]["pos"]}
                rooms[room_code]["connections"]["player2"] = conn

                for player in rooms[room_code]["connections"]:
                    rooms[room_code]["connections"][player].sendall(json.dumps(rooms[room_code]["game_state"]).encode())

                threading.Thread(target=handle_client, args=(conn, addr, room_code, "player2")).start()
            else:
                conn.sendall(json.dumps({"error": "Room full or doesn't exist"}).encode())

if __name__ == "__main__":
    start_server()


In [ ]:
import socket
import json
import pygame
import sys

# Constants
SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600
CELL_SIZE = 20
WHITE, BLACK, GREEN, RED, YELLOW, BLUE = (255, 255, 255), (0, 0, 0), (0, 255, 0), (255, 0, 0), (255, 255, 0), (0, 0, 255)

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Multiplayer Maze Game")

# Connect to the server
server_ip = input("Enter server IP: ")
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect((server_ip, 5555))

# Choose to create or join a room
action = input("Type 'create' to start a game or 'join' to enter a room: ").strip().lower()
if action == "create":
    client.sendall(json.dumps({"action": "create"}).encode())
    response = json.loads(client.recv(4096).decode())
    room_code = response["room_code"]
    print(f"Room created! Share this code: {room_code}")
elif action == "join":
    room_code = input("Enter the room code: ")
    client.sendall(json.dumps({"action": "join", "room_code": room_code}).encode())

# Receive game state
game_state = json.loads(client.recv(4096).decode())
if "error" in game_state:
    print(game_state["error"])
    client.close()
    sys.exit()

maze = game_state["maze"]
players = game_state["players"]
gold_coins = game_state["gold_coins"]
diamonds = game_state["diamonds"]
end_point = game_state["end_point"]

# Get player ID
player_id = "player1" if "player2" not in players else "player2"
print(f"You are {player_id}")

# Function to draw the maze
def draw_maze():
    screen.fill(WHITE)
    for row in range(len(maze)):
        for col in range(len(maze[row])):
            x, y = col * CELL_SIZE, row * CELL_SIZE
            if maze[row][col] == 1:
                pygame.draw.rect(screen, BLACK, (x, y, CELL_SIZE, CELL_SIZE))
            elif (row, col) == (1, 0):  # Start point
                pygame.draw.rect(screen, GREEN, (x, y, CELL_SIZE, CELL_SIZE))
            elif (row, col) == end_point:  # End point
                pygame.draw.rect(screen, RED, (x, y, CELL_SIZE, CELL_SIZE))
    
    # Draw gold coins
    for gx, gy in gold_coins:
        pygame.draw.circle(screen, YELLOW, (gy * CELL_SIZE + CELL_SIZE // 2, gx * CELL_SIZE + CELL_SIZE // 2), CELL_SIZE // 3)

    # Draw diamonds
    for dx, dy in diamonds:
        pygame.draw.circle(screen, BLUE, (dy * CELL_SIZE + CELL_SIZE // 2, dx * CELL_SIZE + CELL_SIZE // 2), CELL_SIZE // 3)

    # Draw players
    for p_id, data in players.items():
        px, py = data["pos"]
        color = (0, 0, 255) if p_id == "player1" else (255, 165, 0)
        pygame.draw.circle(screen, color, (py * CELL_SIZE + CELL_SIZE // 2, px * CELL_SIZE + CELL_SIZE // 2), CELL_SIZE // 2)

# Main game loop
running = True
while running:
    draw_maze()
    pygame.display.flip()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        elif event.type == pygame.KEYDOWN:
            move = None
            if event.key == pygame.K_UP:
                move = "UP"
            elif event.key == pygame.K_DOWN:
                move = "DOWN"
            elif event.key == pygame.K_LEFT:
                move = "LEFT"
            elif event.key == pygame.K_RIGHT:
                move = "RIGHT"

            if move:
                client.sendall(json.dumps({"move": move}).encode())

    # Receive updates from the server
    try:
        data = client.recv(1024).decode()
        if data:
            game_state = json.loads(data)
            players = game_state["players"]
            if "message" in game_state:
                print(game_state["message"])
                running = False
    except:
        pass

pygame.quit()
client.close()
